# 2021/12/21 （鈴木・第３回）集合の扱い方：集合型とビット表現

今日は「集合」を２通りのやり方で扱う。

## 集合型
Python では集合型により有限集合をあつかうことができる。
- https://docs.python.org/ja/3/tutorial/datastructures.html#sets
- https://docs.python.org/ja/3/library/stdtypes.html#set

集合型では、元の追加・削除、合併、差、属しているか判定、などの計算を効率的に行える。

初期化は以下のいずれかで行う
- `set()` という関数で初期化（空集合）
- 波括弧を使って `s = {1,2}` 
- 内包表記を使う `s = {i**2 for i in range(4)}`


In [1]:
s = set() # 初期化
s.add(1) # 要素 1 を追加
s.add(2)
print(s)
print(1 in s) # 1 は s の要素か判定
s.remove(1) # 要素 1 を取り除く
print(s)
print(1 in s) # 1 は s の要素か判定

{1, 2}
True
{2}
False


In [2]:
s = {1,2,3,4}
t = {i**2 for i in range(-4,4)} # 集合なので複数回でてきても１回のみカウント
print(s,t)
print(s&t) # 共通部分
print(s|t) # 合併
print(s-t) # 差
print(s^t) # 対称差


{1, 2, 3, 4} {0, 1, 4, 9, 16}
{1, 4}
{0, 1, 2, 3, 4, 9, 16}
{2, 3}
{0, 16, 2, 3, 9}


### in に関する注意

リストに対しても `in` を使い「リストのなかにあるか判定」ができる。
しかし、リストの `in` は、「リストの元を全て見て、一致するか確認する」という動作原理なので、多数の `in` を実行すると遅くなってしまう。


In [3]:
a = [1,2,3]
print(0 in a) # False
print(1 in a) # True

False
True


In [4]:
a = list(range(10000)) # リスト
cnt = 0
for i in range(5000,35000):
    if i in a:
        cnt += 1
print(cnt)

5000


一方で、集合型は内部的にはハッシュテーブルというものを使った実装がされている。
よって `in` の判定は極めて高速である。

つまり、`in` の判定を大量にする必要がある場合は `set` を使おう！

In [6]:
a = set(range(10000)) # 集合
cnt = 0
for i in range(5000,35000):
    if i in a:
        cnt += 1
print(cnt)

5000


## 辞書型

https://docs.python.org/ja/3/tutorial/datastructures.html#dictionaries

辞書型（連想配列ということもある）を使うと、「キーと値の対応」を保存できる。
たとえば、「名前に電話番号を対応させる」とか。

リストだと 「$0,1,2,\dots$ に対して値を対応させる」ができるが、辞書型はより汎用的である（その分、速度やメモリ効率は犠牲になる）。

さらに、集合と同じように `in` や「キーにアクセスする」が高速にできる。


In [8]:
# 名前に部屋番号を対応させる
name_to_room = {"SUZUKI":"C805", "NOZAKI":"C805", "JIMU":"B709"}
print(name_to_room["SUZUKI"])

C805


In [9]:
# 辞書を初期化して、値をセットしていく例
d = dict()
d[1] = 2
d[2] = 3
print(d[1])
d[1] += 6 # 値を変更
print(d[1])

2
8


In [10]:
d = {1:2, 3:4}
print(d[5]) # キーがセットされていないのでエラーとなる

KeyError: ignored

## ビットによる集合の表現

ここからは、集合型を使わずに集合を表現する方法を学んでいこう。

以下の写像で、「$n$ 元集合の部分集合」と「$0$ 以上 $2^n$ 未満の整数」を対応させる。
つまり、$2$ 進法表記を考えて、$i \in S$ なら下から $i$ 桁目を $1$ にする。

$$
\mathcal{P}({0,1, \dots, n-1}) \to \{0,1, \dots, 2^n-1\}, \qquad S \to \sum_{i \in S} 2^i
$$

つまり、たとえば $n=3$ だと以下のような感じ

|  $\{0,1,2\}$ の部分集合  | 整数の表現 | （先頭にゼロ埋めした）二進数の表現|
| ---- | ---- | ---- |
| $\{\}$      | 0 | 000 |
| $\{0\}$     | 1 | 001 |
| $\{1\}$     | 2 | 010 |
| $\{0,1\}$   | 3 | 011 |
| $\{2\}$     | 4 | 100 |
| $\{0,2\}$   | 5 | 101 |
| $\{1,2\}$   | 6 | 110 |
| $\{0,1,2\}$ | 7 | 111 |

整数の二進数表現が見たい場合、`bin()` という関数が使える。ただし
- `bin()` の返り値は文字列
- 二進数表現を意味する`0b` が頭につく（`s` のかわりに `s[2:]` とすると２文字目以降を取り出せる。結果 `0b` を取り除ける）
- 二進数表現なので、先頭のゼロ埋めはない（ゼロ埋めしたいなら `zfill()` などをつかう）


In [11]:
for i in range(8):
    print(bin(i))

0b0
0b1
0b10
0b11
0b100
0b101
0b110
0b111


In [12]:
for i in range(8):
    print(bin(i)[2:].zfill(3))

000
001
010
011
100
101
110
111


桁数を制限しない場合、以下の２つの集合の間の全単射がある
- $A$: 自然数の集合の有限部分集合（ただしリストで与えられる）
- $\mathbb{N}$　（ここでは $0$ 以上の整数の集合。$0$ は空集合に対応する。）

数学的には、全単射 $f,g$ は以下で与えられる。
$$
f(S) = \sum_{i \in S} 2^i, \qquad g(x) = \{i \in \mathbb{N} \mid x の 2 進法表記の下から i 桁目が立っている \}
$$

プログラムとしては、たとえば以下のコードで実現できる。


In [13]:
def f(S):
    r = 0
    for i in S:
        r += 1<<i  # 2**i としてもよいが、こちらのほうが効率的
    return r

def g(x):
    r = []
    c = 0
    while x > 0:   # x の下の桁から見るのは、２で割り続けながら 2 で割った余りを見ることで実現
        if x%2 == 1:
            r.append(c)
        x //= 2
        c += 1
    return r

print(f([0,2,3]))
print(g(13))


13
[0, 2, 3]


### 注意
集合と整数を対応させる方法は、メモリの効率がよく、配列でデータを管理できるといったメリットがある。一方で、集合と整数を対応させる方法は、要素数が十分小さい集合に限ると思ったほうが良い。

たとえばすべての部分集合を見たい場合、 $20$ 元集合くらいでしんどくなってくる ($2^{20} \simeq 10^6$)。

またそうでなくても、コンピュータで効率的に扱える整数の範囲も限られている。たとえば C 言語などでは unsigned long long 型により $2^{64}$ 未満の非負整数を効率的に扱えるが、これを集合とみなすと $64$ 元集合である。pythonは多倍長整数が扱えるが、効率的に扱える整数の範囲は $2^{62}-1$ ぐらいまでである。



### 整数のビット演算

いま集合と整数を対応させた。しかし、合併、共通部分といった集合演算が整数の世界でも高速にできるだろうか？じつは整数のビット演算により集合演算が高速に行える。

|  集合演算  | `set`での演算 |ビット演算|ビット演算の意味|
| ---- | ---- | ---- ||
| $A \cap B$  | A&B | A&B |両方とも 1 なら 1|
| $A \cup B$ | A\|B | A\|B |どっちかが 1 なら 1|
| $A,B$ の対称差 | A^B | A^B | ちょうど一つが 1 なら 1| 
| $i \in A$ | i in A | (A>>i)&1 | i ビット目が立っているか？|
| $A$ に $i$ を追加 | A.add(i) | A \|= (1<<i) | i ビット目に 1 を立てる|
| $A$ から $i$ を削除 | A.remove(i) | A &= ~(1<<i) | i ビット目の 1 を消す|

ここで、`<<` と `>>` はビットシフト演算子である。

- `a<<2` とすると、a のビット表現が左に２つずれる
- `a>>2` とすると、a のビット表現が右に２つずれる（右の２つは消え去る）

2進法をよく知っている皆さんなら、ビットシフトは数学的には２ベキ倍もしくは２ベキで割ることと同値であることに気付くだろう。

- `a<<2` は `a *= 4` とおなじ
- `a>>2` は `a //= 4` とおなじ

`~` は各ビットを反転する演算子である。
たとえば `~(1<<i)` は、$i$ ビット目以外が $1$ となる数なので、`x & ~(1<<i)` とすることで、$x$ の $i$ ビット目のみを $0$ にすることができる。


In [14]:
a,b = 3,5
print(bin(a)[2:].zfill(3))
print(bin(b)[2:].zfill(3))
print(a&b) # bitwise-and
print(a|b) # bitwise-or
print(a^b) # bitwise-xor
print(b<<2)
print(b>>2)

011
101
1
7
6
20
1


## 練習問題

### 練習問題１

集合 $S_n$, $T_n$ を以下で定める。（平方数の集合と三角数の集合）
- $S_n = \{x^2 \mid x \in \mathbb{Z},\, 0 \le x \le n \}$,
- $T_n = \{x(x+1)/2 \mid x \in \mathbb{Z},\, 0 \le x \le n \}$,

問題
- 準備: $S_n$, $T_n$ を内包表記を使って集合型で表現しよう。
- $n$ を与えると $S_n \cap T_n$ の元をすべて返すような関数 $f(n)$ を作ろう。
- そして $f(1681)$ が $6$ 元集合であることを確かめよう。


In [ ]:
### ここに書いてみよう







**解答例**

In [ ]:
def f(n):
    S = {x**2 for x in range(n+1)}
    T = {x*(x+1)//2 for x in range(n+1)}
    return S&T

f(1681)

{0, 1, 36, 1225, 41616, 1413721}

### 練習問題２

$5$ 元集合 $\{0,1,2,3,4\}$ の部分集合 $S,T$ を
$S = \{0,2,3,4\}$, $T = \{1,3 \}$ と定める。

- $S,T$ に対応する整数 $s,t$ が $s = 29$, $t = 10$ であることを納得せよ。
- $S \cap T$, $S \cup T$ に対応する整数をビット演算を使って求めよ
- 集合論的に $S \cap T$, $S \cup T$ を手計算し、上で計算した整数と対応することを納得せよ。


In [ ]:
# ここに書いてみよう





**解答例**

In [ ]:
s,t = 29,10
print(s&t)
print(s|t)


8
31


### 練習問題３

関数$f(x,i)$ を、 $x$ の ２進法表記の $i$ ビット目（下から $i$ 桁目）が 0 か 1 かを返す関数とする。
関数 $f(x,i)$ を具体的なプログラムとして実現せよ。

たとえば、$6$ の ２進法表記は $110$ なので、$f(6,0) = f(6,3) = 0$, $f(6,1) = f(6,2) = 1$ となることを確認せよ。




In [ ]:
## ここに書いてみよう












**解答例１**

$i$ 桁右にビットシフトすれば、$i$ 桁目が最下位ビットにくる。あとは 1 でビットマスクをすれば $i$ 桁目のみが取り出せる。


In [ ]:
def f(x,i):
    return (x>>i)&1

for i in range(4):
    print(f(6,i))

0
1
1
0


**解答例２**

素直に、i 桁目のビットが立っているか調べる例

In [ ]:
def f(x,i):
    if x&(1<<i) != 0:
        return 1
    else:
         return 0


for i in range(4):
    print(f(6,i))

0
1
1
0


### 練習問題４

関数$f(x)$ を、 $x$ の ２進法表記の $0$ ビット目と $2$ ビット目を $1$ に変える操作をする関数とする。
関数 $f(x)$ を具体的なプログラムとして実現せよ。

たとえば以下の例を確認せよ。
- $f(3) = 7$
- $f(5) = 5$
- $f(8) = 13$
- $f(12) = 13$
- $f(15) = 15$




In [ ]:
## ここにかいてみよう












**解答例**

この解答例のように、ビット演算を活用して必要なビットをまとめて操作することをビットマスクという。

In [ ]:
def f(x):
    return x|5  # 5 の2進法表記は 101

for x in [3,5,8,12,15]:
    print(f(x))

7
5
13
13
15


### 練習問題５

ビット演算だけでいろいろな動作を行うことができる。ここではその黒魔術（ではなく、数学的な根拠はもちろんある）のひとつを見る。

正の数 $x$ に対して `x & (x-1)` はどのような数になっているか？
実験して $x$ の 2進法表記と `x & (x-1)` の2進法表記を比べるなどして考察してみよう。


In [ ]:
## ここで実験してみよう

def f(x):
    return x&(x-1)








**解答例**

ネ

タ

バ

レ

防

止

${}$

${}$

**答え:** $x$ の $2$ 進法表記の「一番下の桁の１」を０にするという操作。
